#### Q1 build_hash

For running Elastic Search 8.17.6:  

```zsh

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    elasticsearch:8.17.6
```

gives "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"


In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
query = 'How do execute a command on a Kubernetes pod?'

In [4]:
from tqdm.auto import tqdm

In [5]:
# from elasticsearch import Elasticsearch
# es_client = Elasticsearch('http://localhost:9200')

from opensearchpy import OpenSearch
es_client = OpenSearch('http://localhost:9200')

es_client.info()

{'name': 'ca3520459c8d',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'sPDGXxpOQDyEMkM1DamC7A',
 'version': {'number': '8.17.6',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7',
  'build_date': '2025-04-30T14:07:12.231372970Z',
  'build_snapshot': False,
  'lucene_version': '9.12.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

#### Q2 indexing

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_settings) # toggled since index already exists

In [11]:
# es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings) # toggled since index already exists

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [12]:
for doc in tqdm(documents):
    es_client.index(
        index=index_name,
        body=doc
    )

  0%|          | 0/948 [00:00<?, ?it/s]

In [13]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # weighting
                    "type": "best_fields"
                }
            },
            # "filter": {
            #     "term": {
            #         "course": "machine-learning-zoomcamp"
            #         }
            #     }
            }
        }
    }

response = es_client.search(
    index=index_name,
    body=search_query
)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

#### Q3 searching

In [14]:
# 3. Loop through the results and extract the score and source
print(f"Found {response['hits']['total']['value']} results.\n")

scores = []

for hit in response['hits']['hits']:
    score = hit['_score']
    scores.append(score)
    source_data = hit['_source']
    
    # Now you can use them
    print(f"Score: {score:.4f}") # Format to 4 decimal places for readability
    print(f"  Question: {source_data.get('question', 'N/A')}")
    print(f"  Course: {source_data.get('course', 'N/A')}")
    print("-" * 30)

Found 739 results.

Score: 44.5056
  Question: How do I debug a docker container?
  Course: machine-learning-zoomcamp
------------------------------
Score: 35.4334
  Question: Kubernetes-dashboard
  Course: machine-learning-zoomcamp
------------------------------
Score: 33.7097
  Question: How do I copy files from a different folder into docker container’s working directory?
  Course: machine-learning-zoomcamp
------------------------------
Score: 33.2635
  Question: How to run a script while a web-server is working?
  Course: machine-learning-zoomcamp
------------------------------
Score: 32.5891
  Question: How can I annotate a graph?
  Course: machine-learning-zoomcamp
------------------------------


In [15]:
scores.sort(reverse=True)
scores[:5]  # Top 5 scores

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

#### Q4 filtering

In [16]:
query= 'How do copy a file to a Docker container?'
query

'How do copy a file to a Docker container?'

In [21]:
def f_elasticsearch(query):
    search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # weighting
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(
        index=index_name,
        body=search_query
        )

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [22]:
result_docs = f_elasticsearch(query)

In [23]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

#### Q5 prompt

In [ ]:
# testing context generation
context = ""
for doc in result_docs:
    context += f"Q: {doc['question']}\nA: {doc['text']}\n\n"
print(context)

In [20]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT:
        {context}
        """.strip() 

    context = ""
    for doc in search_results:
        context += f"Q: {doc['question']}\nA: {doc['text']}\n\n"

    prompt = prompt_template.format(
        question=query,
        context=context
        ).strip()

    return prompt

In [25]:
prompt = build_prompt(query, result_docs)

In [24]:
query, len(result_docs)

('How do copy a file to a Docker container?', 3)

In [26]:
len(prompt)

1478

#### Q6

##### tokens count with tiktoken

In [29]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
len(tokens)

324

##### tokens count with response.usage_metadata

In [30]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Example: get a variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')


In [31]:
from google import genai
from google.genai import types

In [32]:
# Only run this block for Gemini Developer API
client = genai.Client(api_key=GOOGLE_API_KEY)

In [33]:
def llm(query, prompt):
    client = genai.Client(api_key=GOOGLE_API_KEY)

    response = client.models.generate_content(
        model='gemini-2.5-pro', 
        contents=[query, prompt]
    )
    
    return response.text.strip()

In [34]:
def rag(query):
    """
    RAG: Retrieve, Augment, Generate
    """

    search_results = f_elasticsearch(query)
    prompt = build_prompt(query, search_results)
    answer = llm(query, prompt) 
    # answer = openai_llm(prompt) 

    return answer

In [35]:
answer = rag(query)

In [36]:
answer

'Based on the context provided, here are two ways to copy a file to a Docker container:\n\n**Method 1: Using the `docker cp` command**\n\nYou can copy files from your local machine into a running Docker container. The command syntax is:\n`docker cp /path/to/local/file_or_directory container_id:/path/in/container`\n\n**Method 2: Using a `Dockerfile`**\n\nYou can copy files from a different folder into the container’s working directory when building the image. In the Dockerfile, use the `COPY` command with the following syntax:\n`COPY ["src/predict.py", "models/xgb_model.bin", "./"]`'

In [37]:
response = client.models.generate_content(
        model='gemini-2.5-pro', 
        contents=[query, prompt])

In [38]:
# Access the usage metadata from the response
usage_data = response.usage_metadata

print("--- Token Usage ---")
print(f"Prompt Tokens (Input): {usage_data.prompt_token_count}")
print(f"Candidates Tokens (Output): {usage_data.candidates_token_count}")
print(f"Total Tokens: {usage_data.total_token_count}")

--- Token Usage ---
Prompt Tokens (Input): 369
Candidates Tokens (Output): 129
Total Tokens: 1650
